In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import nbinom, poisson
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
import seaborn as sns

In [2]:
data = pd.read_csv("Large_Trucks_MNL_Dataset.csv")
data

,INJSVR,TOTUNIT,INTSEC,INTSEC_TRFCONT,ALCFLAG,DRUGFLAG,YOUNG,OLD,FEMALE,RURAL,...,TRFCONT_SIGNAL,TRFCONT_2WAY,TRFCONT_4WAY,TRFCONT_YIELD,TRFCONT_NONE,TRUCK_TRUCK,TRUCK_PC,TRUCK_BMP,TRUCK_OTHERS,COLLTYPE
0,2,2,1,1,0,0,0,0,0.0,1,...,0,0,0,0,1,1,0,0,0,1
1,2,3,0,0,0,1,0,0,0.0,1,...,0,0,0,0,1,0,1,0,0,2
2,1,3,1,1,0,0,0,0,0.0,0,...,0,1,0,0,0,0,1,0,0,2
3,1,2,0,0,0,0,0,1,0.0,0,...,0,0,0,0,1,0,1,0,0,2
4,1,2,0,0,0,0,1,0,0.0,1,...,0,0,0,0,1,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,2,0,0,0,0,0,0,0.0,1,...,0,0,0,0,1,1,0,0,0,1
9996,1,2,0,0,0,0,0,0,0.0,1,...,0,0,0,0,1,1,0,0,0,1
9997,1,2,0,0,0,0,0,0,0.0,1,...,0,0,0,0,1,1,0,0,0,1
9998,1,2,0,0,0,0,0,0,0.0,1,...,0,0,0,0,1,1,0,0,0,1


In [4]:
data.head(2).T

,0,1
INJSVR,2.0,2.0
TOTUNIT,2.0,3.0
INTSEC,1.0,0.0
INTSEC_TRFCONT,1.0,0.0
ALCFLAG,0.0,0.0
DRUGFLAG,0.0,1.0
YOUNG,0.0,0.0
OLD,0.0,0.0
FEMALE,0.0,0.0
RURAL,1.0,1.0


In [5]:
data.INJSVR.unique()

array([2, 1, 3])

In [6]:
data.INJSVR.value_counts(normalize = True) * 100

1    49.05
2    39.81
3    11.14
Name: INJSVR, dtype: float64

In [7]:
#Setup Binary Problem(Second Group)

In [8]:
data['injury_binary'] = [0 if i==1 else 1 for i in data.INJSVR]
data[['INJSVR', 'injury_binary']].head(1)

,INJSVR,injury_binary
0,2,1


In [9]:
data.columns

Index(['INJSVR', 'TOTUNIT', 'INTSEC', 'INTSEC_TRFCONT', 'ALCFLAG', 'DRUGFLAG',
       'YOUNG', 'OLD', 'FEMALE', 'RURAL', 'SAFETY', 'WTHRCOND_CLOUDY',
       'WTHRCOND_FOG', 'WTHRCOND_RAIN', 'WTHRCOND_SLEET', 'WTHRCOND_SNOW',
       'WTHRCOND_WIND', 'LGTCOND_DARK', 'LGTCOND_LIGHT', 'ROADCOND_ICE',
       'ROADCOND_SNOW', 'ROADCOND_WET', 'MNRCOLL', 'MNRCOLL_ANGLE',
       'MNRCOLL_HEAD', 'MNRCOLL_REAR', 'MNRCOLL_SSO', 'MNRCOLL_SSS',
       'DRVRPC_SPD', 'DRVRPC_RULEVIO', 'DRVRPC_RECK', 'HWYPC_PDB',
       'HWYPC_SHOULDER', 'HWYPC_VIS', 'ACCDTYPE_BIKE', 'ACCDTYPE_BRIDGE',
       'ACCDTYPE_DEER', 'ACCDTYPE_DITCH', 'ACCDTYPE_GUARDRAIL',
       'ACCDTYPE_JACKKNIFE', 'ACCDTYPE_MEDBAR', 'ACCDTYPE_OVERTURN',
       'ACCDTYPE_PARKVEH', 'ACCDTYPE_TREE', 'ACCDTYPE_POLE', 'TRFCONT_1',
       'TRFCONT', 'TRFCONT_SIGNAL', 'TRFCONT_2WAY', 'TRFCONT_4WAY',
       'TRFCONT_YIELD', 'TRFCONT_NONE', 'TRUCK_TRUCK', 'TRUCK_PC', 'TRUCK_BMP',
       'TRUCK_OTHERS', 'COLLTYPE', 'injury_binary'],
      dtype='obj

In [10]:
data.MNRCOLL.unique()

array([ 1., nan,  2.,  6.,  3.])

In [14]:
data['manner_col'] = data.MNRCOLL -1

In [15]:
formula = 'injury_binary ~ TOTUNIT + INTSEC + DRUGFLAG + OLD + FEMALE + C(MNRCOLL)'
model = smf.glm(formula = formula, data = data, family = sm.families.Binomial(sm.families.links.Logit())).fit()

In [16]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          injury_binary   No. Observations:                 9796
Model:                            GLM   Df Residuals:                     9787
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -6467.4
Date:                Tue, 01 Nov 2022   Deviance:                       12935.
Time:                        21:19:20   Pearson chi2:                 1.00e+04
No. Iterations:                     5   Pseudo R-squ. (CS):            0.06302
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.9660      0.065    -14.752      0.000      -1.094      -0.838
C(MNRCOLL)[T.2.0]    -0.1769      0.098     -1.803      0.071      -0.369       0.015
C(MNRCOLL)[T.3.0]    -0.0704      0.059     -1.202      0.230      -0.185       0.044
C(MNRCOLL)[T.6.0]     0.0289      0.048      0.603      0.547      -0.065       0.123
TOTUNIT               0.3482      0.026     13.269      0.000       0.297       0.400
INTSEC                0.3217      0.044      7.256      0.000       0.235       0.409
DRUGFLAG              2.0799      0.383      5.437      0.000       1.330       2.830
OLD                   0.0358      0.053      0.677      0.498      -0.068       0.139
FEMALE                0.7655      0.050     15.441      0.000       0.668       0.863
=====================================================================================
"""

In [17]:
np.exp(model.params)

Intercept            0.380591
C(MNRCOLL)[T.2.0]    0.837831
C(MNRCOLL)[T.3.0]    0.931993
C(MNRCOLL)[T.6.0]    1.029281
TOTUNIT              1.416584
INTSEC               1.379424
DRUGFLAG             8.003697
OLD                  1.036440
FEMALE               2.150114
dtype: float64

In [18]:
#np.exp(model.params)#

In [19]:
data.INJSVR.value_counts(normalize = True) * 100

1    49.05
2    39.81
3    11.14
Name: INJSVR, dtype: float64

In [24]:
from statsmodels.discrete.discrete_model import MNLogit

In [28]:
formula = 'INJSVR ~ TOTUNIT + INTSEC + DRUGFLAG + OLD + FEMALE + C(MNRCOLL)'
model_MNL = MNLogit.from_formula(formula , data ).fit()

Optimization terminated successfully.
         Current function value: 0.925824
         Iterations 6


In [30]:
print(model_MNL.summary())

                          MNLogit Regression Results                          
Dep. Variable:                 INJSVR   No. Observations:                 9796
Model:                        MNLogit   Df Residuals:                     9778
Method:                           MLE   Df Model:                           16
Date:                Tue, 01 Nov 2022   Pseudo R-squ.:                 0.03846
Time:                        21:31:23   Log-Likelihood:                -9069.4
converged:                       True   LL-Null:                       -9432.2
Covariance Type:            nonrobust   LLR p-value:                4.563e-144
         INJSVR=2       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.1509      0.068    -16.819      0.000      -1.285      -1.017
C(MNRCOLL)[T.2.0]    -0.1535      0.104     -1.482      0.138      -0.356       0.049
C(MNRCOLL)[T.3.0]    -0.

1 -> PDO

2 -> Possible injury

3 -> Injuries and fatability

In [31]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from patsy import dmatrices

In [32]:
daxf = data.dropna(axis = 0)

In [34]:
mod_prob = OrderedModel(daxf['INJSVR'],
                        daxf[['TOTUNIT', 'INTSEC', 'DRUGFLAG', 'OLD', 'FEMALE']],
                        distr='logit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 0.933806
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                 INJSVR   Log-Likelihood:                -9147.6
Model:                   OrderedModel   AIC:                         1.831e+04
Method:            Maximum Likelihood   BIC:                         1.836e+04
Date:                Tue, 01 Nov 2022                                         
Time:                        21:38:07                                         
No. Observations:                9796                                         
Df Residuals:                    9789                                         
Df Model:                           7                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
TOTUNIT        0.2849      0.021     13.588      0.000       0.244       0.326
INTSEC         0.3352      0.041      8.092      0.000       0.254       0.416
DRUGFLAG       2.0057      0.244      8.233      0.000       1.528       2.483
OLD            0.1445      0.050      2.897      0.004       0.047       0.242
FEMALE         0.6046      0.044     13.603      0.000       0.518       0.692
1/2            0.8454      0.052     16.353      0.000       0.744       0.947
2/3            0.7969      0.015     53.765      0.000       0.768       0.826
==============================================================================
"""

In [35]:
print(res_prob.summary())

                             OrderedModel Results                             
Dep. Variable:                 INJSVR   Log-Likelihood:                -9147.6
Model:                   OrderedModel   AIC:                         1.831e+04
Method:            Maximum Likelihood   BIC:                         1.836e+04
Date:                Tue, 01 Nov 2022                                         
Time:                        21:38:32                                         
No. Observations:                9796                                         
Df Residuals:                    9789                                         
Df Model:                           7                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
TOTUNIT        0.2849      0.021     13.588      0.000       0.244       0.326
INTSEC         0.3352      0.041      8.092      0.0